# Dependencies

In [22]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
import pandas as pd

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
 # Define database and collection
db = client.mars_db
collection = db.articles

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

# Scraping

## NASA Mars News

In [20]:
# Retrieve the latest news title
news_title = soup.find_all('div', class_='content_title')[0].text

# print(news_title)

# Retrieve the latest news paragraph
news_paragraph = soup.find_all('div', class_='rollover_description_inner')[0].text

print(f"The latest News Title and Paragraph Text from Nasa's Mars Exploration website is {news_title} {news_paragraph}")

The lates News Title and Paragraph Text from Nasa's Mars Exploration website is 

NASA's Next Mars Rover Is Ready for the Most Precise Landing Yet

 
What to expect when the Mars 2020 Perseverance rover arrives at the Red Planet on Feb. 18, 2021.



## JPL Mars Space Images - Featured Image

In [47]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/lhabersham/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [48]:
jpl_url = 'https://www.jpl.nasa.gov/images?search=&category=Mars'
browser.visit(jpl_url)

In [49]:
for x in range(1, 2):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    imgs = soup.find_all('div', class_='SearchResultCard')[0]

# Use Beautiful Soup's find() method to navigate and retrieve attributes
    for img in imgs:
        link = imgs.find('a')
        href = link['href']
        print('-----------')
        print(img.text)
        print('https://www.jpl.nasa.gov' + href)

-----------

        Image
        .   
          Image
          .   Great Anticipation as Perseverance Lands  Feb 19, 2021  
https://www.jpl.nasa.gov/images/great-anticipation-as-perseverance-lands


In [50]:
featured_image_url = 'https://www.jpl.nasa.gov/images/great-anticipation-as-perseverance-lands'
print(featured_image_url)

https://www.jpl.nasa.gov/images/great-anticipation-as-perseverance-lands


## Mars Facts

In [51]:
mars_facts_url = 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)

In [55]:
mars_fact = pd.read_html(mars_facts_url)
mars_fact

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [63]:
#Find Mars Facts DataFrame in the lists of DataFrames
mars_facts = mars_fact[0]
# mars_facts

#Assign the columns
mars_facts.columns = ['Description', 'Value']
# mars_facts

mars_facts

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [64]:
mars_facts.to_html('mars_facts.html', index=False, classes=['table', 'table-striped', 'table-hover'])

In [65]:
print(mars_facts)

            Description                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


In [ ]:
browser.quit()

In [15]:
# Retrieve the parent divs for all News Titles and Paragraph Texts
results = soup.find_all('div', class_='content_title')
print(results)


# loop over results to get News Titles and Paragraph Texts
for result in results:
    # scrape the News Titles 
    news_titles = result.find('a', target_ ='self').text
    
    # scrape the Paragraph Texts
    paragraph_texts = result.find('div', class_='article_teaser_body').text
    
    # News Titles and Paragraph Texts
    print('-----------------')
    print(news_titles)
    print(paragraph_texts)

#     # Dictionary to be inserted into MongoDB
#     post = {
#         'header': header,
#         'subheader': subheader,
#         'date': date,
#     }

#     # Insert dictionary into MongoDB as a document
#     collection.insert_one(post)

[<div class="content_title">
<a href="/news/8860/nasas-next-mars-rover-is-ready-for-the-most-precise-landing-yet/">
NASA's Next Mars Rover Is Ready for the Most Precise Landing Yet
</a>
</div>, <div class="content_title">
<a href="/news/8837/nasa-to-host-virtual-briefing-on-february-perseverance-mars-rover-landing/">
NASA to Host Virtual Briefing on February Perseverance Mars Rover Landing
</a>
</div>, <div class="content_title">
<a href="/news/8836/nasa-insights-mole-ends-its-journey-on-mars/">
NASA InSight's ‘Mole' Ends Its Journey on Mars
</a>
</div>, <div class="content_title">
<a href="/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/">
NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities
</a>
</div>, <div class="content_title">
<a href="/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/">
The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
</a>
</div>, <div class="content_title">
<a href="/ne

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

sidebar = soup.find('ul', class_='nav-list')

categories = sidebar.find_all('li')

category_list = []
url_list = []
book_url_list = []
for category in categories:
    title = category.text.strip()
    category_list.append(title)
    book_url = category.find('a')['href']
    url_list.append(book_url)

book_url_list = ['http://books.toscrape.com/' + url for url in url_list]

titles_and_urls = zip(category_list, book_url_list)

try:
    for title_url in titles_and_urls:
        browser.links.find_by_partial_text('next').click()
except ElementDoesNotExist:
    print("Scraping Complete")